In [ ]:
import math, os, sys
import json
import ast
import plotly.express as px
import pandas as pd
from datetime import datetime
sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [ ]:
from neuroglancer.models import UrlModel, LayerData, CenterOfMass, Structure
from brain.models import Animal

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

In [ ]:
id = 211
id = 214
urlModel = UrlModel.objects.get(pk=id)
prep_id = urlModel.animal
prep = Animal.objects.get(pk=prep_id)

In [ ]:
prep.prep_id

In [ ]:
point_data = find_values('annotations', urlModel.url)

In [ ]:
json_txt = json.loads(urlModel.url)
layers = json_txt['layers']

In [ ]:
for layer in layers:
    if 'annotations' in layer:
        annotation = layer['annotations']
        d = [row['point'] for row in annotation]
        if 'com' in annotation:
            print(annotation)


In [ ]:
id = 211
id = 214
urlModel = UrlModel.objects.get(pk=id)

result = None
dfs = []
description = None
json_txt = json.loads(urlModel.url)
layers = json_txt['layers']
for layer in layers:
    if 'annotations' in layer:
        name = layer['name']
        annotation = layer['annotations']
        d = [row['point'] for row in annotation]
        df = pd.DataFrame(d, columns=['X', 'Y', 'Section'])
        df['X'] = df['X'].astype(int)
        df['Y'] = df['Y'].astype(int)
        df['Section'] = df['Section'].astype(int)
        df['Layer'] = name
        structures = [row['description'] for row in annotation if 'description' in row]
        if len(structures) != len(df):
            structures = ['' for row in annotation]
        df['Description'] = structures
        df = df[['Layer', 'Description', 'X', 'Y', 'Section']]
        dfs.append(df)
if len(dfs) == 0:
    result = None
elif len(dfs) == 1:
    result = dfs[0]
else:
    result = pd.concat(dfs)
result.head()

In [ ]:
structure = Structure.objects.get(abbreviation='Pn')
print(structure.id)

In [ ]:
reference_centers = sql_controller.get_centers_dict(prep.prep_id)
atlas_box_size=(2010,1490,  447)
align_atlas(reference_centers, reference_scales=(0.00001, 0.00001, 20))

In [ ]:
#df = px.data.iris()
fig = px.scatter_3d(df, x='X', y='Y', z='Section',
              color='Section', opacity=0.7)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()